# Análise Exploratória de Dados

Este notebook executa uma análise exploratória completa no dataset `Obesity.csv`, para analisar os dados antes de prepará-los para o feature engineering.



In [9]:
from __future__ import annotations

from pathlib import Path
from typing import Iterable

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt


sns.set_theme(style="whitegrid")


def detectar_raiz_projeto() -> Path:
    """Localizar a raiz do projeto, que contém o dataset."""
    caminho_atual = Path().resolve()
    for candidato in [caminho_atual, *caminho_atual.parents]:
        arquivo = candidato / "data" / "Obesity.csv"
        if arquivo.exists():
            return candidato
    raise FileNotFoundError(
        "Não foi possível localizar o arquivo 'Obesity.csv' na hierarquia de diretórios."
    )


PROJECT_ROOT = detectar_raiz_projeto()
DATA_PATH = PROJECT_ROOT / "data" / "Obesity.csv"
PLOTS_DIR = PROJECT_ROOT / "plots"

TARGET_COLUMN = "Obesity"
NUMERIC_FEATURES = [
    "Age",
    "Height",
    "Weight",
    "BMI",
    "FCVC",
    "NCP",
    "CH2O",
    "FAF",
    "TUE",
]
CATEGORICAL_FEATURES = [
    "Gender",
    "family_history",
    "FAVC",
    "CAEC",
    "SMOKE",
    "SCC",
    "CALC",
    "MTRANS",
]


In [10]:
def garantir_diretorio_plots(diretorio: Path) -> None:
    """Garantir que o diretório para plots existe."""
    diretorio.mkdir(parents=True, exist_ok=True)


def carregar_dados(caminho: Path) -> pd.DataFrame:
    """Carregar o dataset no df."""
    if not caminho.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {caminho}")
    return pd.read_csv(caminho)


def analise_inicial(df: pd.DataFrame) -> None:
    """Roda checks iniciais no dataset."""
    print("\n===== Informações da base =====")
    df.info()

    print("\n===== Estatísticas descritivas (numéricas) =====")
    print(df.describe())

    print("\n===== Estatísticas descritivas (categóricas) =====")
    categ = df.select_dtypes(include="object").columns
    if len(categ) > 0:
        print(df[categ].describe(include="object"))
    else:
        print("Nenhuma variável categórica encontrada.")

    print("\n===== Valores nulos por coluna =====")
    print(df.isnull().sum())

    print("\n===== Total de linhas duplicadas =====")
    print(df.duplicated().sum())


def criar_bmi(df: pd.DataFrame) -> pd.DataFrame:
    """Criar feature BMI (IMC) usando Weight e Height."""
    resultado = df.copy()
    altura = resultado["Height"].replace({0: np.nan})
    resultado["BMI"] = resultado["Weight"] / (altura**2)
    return resultado


def salvar_figura(nome_arquivo: str) -> None:
    """Salva a figura do matplotlib atual para o diretório de plots."""
    caminho = PLOTS_DIR / nome_arquivo
    plt.tight_layout()
    plt.savefig(caminho, dpi=300)
    plt.close()
    print(f"Gráfico salvo em: {caminho}")


def analisar_alvo(df: pd.DataFrame, coluna_alvo: str = TARGET_COLUMN) -> None:
    """Analisar a distribuição da variável target e salva o plot."""
    if coluna_alvo not in df.columns:
        raise KeyError(f"Coluna alvo '{coluna_alvo}' não encontrada no DataFrame.")

    plt.figure(figsize=(10, 6))
    ordem = df[coluna_alvo].value_counts().index
    sns.countplot(data=df, x=coluna_alvo, order=ordem)
    plt.title("Distribuição da variável alvo")
    plt.xlabel("Nível de obesidade")
    plt.ylabel("Contagem")

    salvar_figura("1_target_distribution.png")

    contagem = df[coluna_alvo].value_counts()
    percentual = contagem / len(df) * 100
    print("\n===== Distribuição do alvo =====")
    for classe, qtd in contagem.items():
        pct = percentual[classe]
        print(f"{classe}: {qtd} registros ({pct:.2f}%)")


In [11]:
def _filtrar_features_existentes(df: pd.DataFrame, features: Iterable[str]) -> list[str]:
    """Filtra nomes das features para as existentes no DF."""
    return [col for col in features if col in df.columns]


def analisar_univariada(df: pd.DataFrame) -> None:
    """Gera plots univariados para features numéricas e categóricas."""
    colunas_numericas = _filtrar_features_existentes(df, NUMERIC_FEATURES)
    for coluna in colunas_numericas:
        plt.figure(figsize=(8, 5))
        sns.histplot(data=df, x=coluna, kde=True, bins=30, color="#1f77b4")
        plt.title(f"Distribuição de {coluna}")
        plt.xlabel(coluna)
        plt.ylabel("Frequência")
        salvar_figura(f"1_univar_{coluna.lower()}_hist.png")

    colunas_categoricas = _filtrar_features_existentes(df, CATEGORICAL_FEATURES)
    for coluna in colunas_categoricas:
        plt.figure(figsize=(10, 6))
        sns.countplot(data=df, x=coluna, color="#ff7f0e")
        plt.title(f"Distribuição de {coluna}")
        plt.xlabel(coluna)
        plt.ylabel("Contagem")
        plt.xticks(rotation=45, ha="right")
        salvar_figura(f"1_univar_{coluna.lower()}_count.png")


def analisar_bivariada(df: pd.DataFrame, coluna_alvo: str = TARGET_COLUMN) -> None:
    """Gera plots bivariados, comparando features com a target."""
    if coluna_alvo not in df.columns:
        raise KeyError(f"Coluna alvo '{coluna_alvo}' não encontrada no DataFrame.")

    ordem_alvo = df[coluna_alvo].value_counts().index

    colunas_numericas = _filtrar_features_existentes(df, NUMERIC_FEATURES)
    for coluna in colunas_numericas:
        if coluna == coluna_alvo:
            continue
        plt.figure(figsize=(10, 6))
        sns.boxplot(data=df, x=coluna_alvo, y=coluna, order=ordem_alvo)
        plt.title(f"{coluna} vs {coluna_alvo}")
        plt.xlabel("Nível de obesidade")
        plt.ylabel(coluna)
        salvar_figura(f"2_bivar_{coluna.lower()}_vs_{coluna_alvo.lower()}.png")

    colunas_categoricas = _filtrar_features_existentes(df, CATEGORICAL_FEATURES)
    for coluna in colunas_categoricas:
        plt.figure(figsize=(12, 6))
        sns.countplot(
            data=df,
            x=coluna,
            hue=coluna_alvo,
            order=df[coluna].value_counts().index,
        )
        plt.title(f"{coluna} vs {coluna_alvo}")
        plt.xlabel(coluna)
        plt.ylabel("Contagem")
        plt.xticks(rotation=45, ha="right")
        plt.legend(title=coluna_alvo, bbox_to_anchor=(1.05, 1), loc="upper left")
        salvar_figura(f"2_bivar_{coluna.lower()}_vs_{coluna_alvo.lower()}.png")


def analisar_correlacao(df: pd.DataFrame) -> None:
    """Gera e salva um mapa de calor de correlação para as features numéricas."""
    colunas_numericas = _filtrar_features_existentes(df, NUMERIC_FEATURES)
    if not colunas_numericas:
        print("Nenhuma coluna numérica disponível para correlação.")
        return

    matriz_corr = df[colunas_numericas].corr()
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        matriz_corr,
        annot=True,
        fmt=".2f",
        cmap="coolwarm",
        square=True,
        cbar_kws={"shrink": 0.8},
    )
    plt.title("Mapa de calor das correlações")
    salvar_figura("3_correlation_heatmap.png")


In [12]:
garantir_diretorio_plots(PLOTS_DIR)
df = carregar_dados(DATA_PATH)
analise_inicial(df)


===== Informações da base =====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2111 entries, 0 to 2110
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Gender          2111 non-null   object 
 1   Age             2111 non-null   float64
 2   Height          2111 non-null   float64
 3   Weight          2111 non-null   float64
 4   family_history  2111 non-null   object 
 5   FAVC            2111 non-null   object 
 6   FCVC            2111 non-null   float64
 7   NCP             2111 non-null   float64
 8   CAEC            2111 non-null   object 
 9   SMOKE           2111 non-null   object 
 10  CH2O            2111 non-null   float64
 11  SCC             2111 non-null   object 
 12  FAF             2111 non-null   float64
 13  TUE             2111 non-null   float64
 14  CALC            2111 non-null   object 
 15  MTRANS          2111 non-null   object 
 16  Obesity         2111 non-null   object 
dtype

In [13]:
df_bmi = criar_bmi(df)
analisar_alvo(df_bmi)

Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_target_distribution.png

===== Distribuição do alvo =====
Obesity_Type_I: 351 registros (16.63%)
Obesity_Type_III: 324 registros (15.35%)
Obesity_Type_II: 297 registros (14.07%)
Overweight_Level_I: 290 registros (13.74%)
Overweight_Level_II: 290 registros (13.74%)
Normal_Weight: 287 registros (13.60%)
Insufficient_Weight: 272 registros (12.88%)


In [18]:
analisar_univariada(df_bmi)
analisar_bivariada(df_bmi)
analisar_correlacao(df_bmi)

Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_age_hist.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_height_hist.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_weight_hist.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_bmi_hist.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_fcvc_hist.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_ncp_hist.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_ch2o_hist.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_faf_hist.png
Gráfico salvo em: C:\Users\Arnaldo Laudares\Documents\FIAP\t4\tech-challenge-4\plots\1_univar_tue_hist.png
Gráfico salvo em: C:\Users\Ar